<a href="https://colab.research.google.com/github/s528661/nlp100/blob/main/nlp100_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 80. ID番号への変換

問題51で構築した学習データ中の単語にユニークなID番号を付与したい．学習データ中で最も頻出する単語に1，2番目に頻出する単語に2，……といった方法で，学習データ中で2回以上出現する単語にID番号を付与せよ．そして，与えられた単語列に対して，ID番号の列を返す関数を実装せよ．ただし，出現頻度が2回未満の単語のID番号はすべて0とせよ．

In [1]:
!pip install mecab-python3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 8.6 MB/s eta 0:00:00


In [2]:
import re,sys,os

import pickle
import time
import copy

import math
import random
import string

from collections import defaultdict, Counter

import numpy as np
import pandas as pd
import scipy as scp

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn import feature_extraction, preprocessing

from sklearn import svm
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, f1_score
from sklearn.metrics import accuracy_score,  precision_score, recall_score, confusion_matrix, roc_auc_score

# import dask.array as da

import lightgbm as lgb

# Neural Language Processing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import SelectKBest

from sklearn.linear_model import RidgeClassifier
import MeCab

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [3]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip --no-check-certificate
!unzip ./NewsAggregatorDataset.zip

--2023-12-08 20:52:14--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas     [        <=>         ]  27.87M  14.8MB/s    in 1.9s    

2023-12-08 20:52:17 (14.8 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203]

Archive:  ./NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


In [39]:
data = pd.read_csv('newsCorpora.csv',delimiter='\t',header=None,
                 names=["ID", "TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"])

In [40]:
fix_data = data[data['PUBLISHER'].isin(["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"])]

In [41]:
def preprocessing(text):
  text = "".join([i for i in text if i not in string.punctuation])
  text = text.lower()
  text = re.sub("[0-9]+", "", text)
  return text

In [42]:
fix_data['pTITLE'] = fix_data['TITLE'].apply(lambda x: preprocessing(x))

<ipython-input-42-0330034268ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fix_data['pTITLE'] = fix_data['TITLE'].apply(lambda x: preprocessing(x))


In [43]:
train_all, test = train_test_split(fix_data, test_size=0.1, shuffle=True)

In [44]:
train, val = train_test_split(train_all, test_size=0.1, shuffle=True)

In [45]:
train.to_csv("train.txt", sep="\t", index=False, header=None)
val.to_csv("valid.txt", sep="\t", index=False, header=None)
test.to_csv("test.txt", sep="\t", index=False, header=None)

In [46]:
def preprocessing(text):
  text = "".join([i for i in text if i not in string.punctuation])
  text = text.lower()
  text = re.sub("[0-9]+", "", text)
  return text

In [47]:
fix_data['pTITLE'] = fix_data['TITLE'].apply(lambda x: preprocessing(x))

<ipython-input-47-0330034268ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fix_data['pTITLE'] = fix_data['TITLE'].apply(lambda x: preprocessing(x))


In [48]:
#vectorizer = TfidfVectorizer(min_df=10, ngram_range=(1, 2)) # 1-gram, 2-gramでTfidfを計算
#X = vectorizer.fit_transform(fix_data['pTITLE']).toarray()
#X = pd.DataFrame(X, columns=vectorizer.get_feature_names_out())
X = fix_data['pTITLE']

In [49]:
X_train_all, X_test = train_test_split(X, test_size=0.1, shuffle=True)

In [50]:
X_train, X_val = train_test_split(X_train_all, test_size=0.1, shuffle=True)

In [51]:
X_train

176094     as data breach woes continue targets ceo resigns
45902     lwren scott is laid to rest in los angeles fun...
210714    olympic champion charlie white gets axed from ...
11264     ed speleers and jesse plemons compete to be je...
363391     miley cyrus  miley cyrus gets tattoo of late dog
                                ...                        
54274     james franco  james franco dismisses lindsay l...
237232              wheres my hammer did you see my blusher
377546        citigroups b settlement who actually benefits
28691     update kocherlakota feds lone dissenter blasts...
291634    kylie and kendall jenner made surprisingly goo...
Name: pTITLE, Length: 10805, dtype: object

In [52]:
X_val

279721    magic school bus to be rebooted by netflix for...
290209    rptrussia cuts gas to ukraine flows to eu thre...
155255    barclays will post small decline in firstquart...
47046     ukraine to hike domestic gas prices by  percen...
262826    us stocks snapshotwall st dips sp ends day str...
                                ...                        
107015                  outkasts reunion had a weird ending
211288    samsung shares steady after chairmans heart at...
377197    global marketsstocks bonds edge up after dragh...
299769                north wests pierced ears spark debate
196085          costco april samestore sales beat estimates
Name: pTITLE, Length: 1201, dtype: object

In [53]:
X_test

332407          tv  kim kardashian househunting in new york
275650    us yields set to spoil ecb easing party for em...
18931     kate winslet  kate winslet receives walk of fa...
364744    sarah palin is publicly auditioning to join th...
230167    rptlenovo aims to sell  mln smartphones tablet...
                                ...                        
189973    with emma stone and joaquin phoenix woody alle...
301030    update cyprus begins marketing june  euro benc...
120416    zebra piles up debt to buy motorola unit for  ...
336304    fail lionel richies name spelled wrong at the ...
76694     us stockswall street to open higher manufactur...
Name: pTITLE, Length: 1334, dtype: object

In [54]:
X_train = X_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [59]:
words = {}
for title in X_train:
  for word in title.rstrip().split():
    words[word] = 0

In [60]:
for title in X_train:
  for word in title.rstrip().split():
    words[word] += 1

In [61]:
id_words = [(idx, item) for idx, item in enumerate(words.items()) if item[1] >= 2]

In [62]:
id_words = sorted(id_words, key=lambda x: x[1][1], reverse=True)

In [63]:
id_words

[(12, ('to', 2895)),
 (14, ('in', 1955)),
 (31, ('the', 1621)),
 (48, ('of', 1476)),
 (131, ('for', 1373)),
 (75, ('on', 1329)),
 (0, ('as', 1164)),
 (79, ('us', 1045)),
 (278, ('update', 1045)),
 (36, ('and', 984)),
 (140, ('a', 832)),
 (30, ('with', 753)),
 (55, ('at', 685)),
 (10, ('is', 627)),
 (198, ('after', 613)),
 (161, ('new', 556)),
 (389, ('says', 418)),
 (28, ('from', 403)),
 (204, ('up', 367)),
 (19, ('by', 331)),
 (446, ('kardashian', 283)),
 (344, ('her', 263)),
 (445, ('kim', 255)),
 (321, ('over', 253)),
 (40, ('be', 236)),
 (660, ('china', 233)),
 (226, ('first', 219)),
 (152, ('stocks', 218)),
 (285, ('more', 214)),
 (256, ('will', 212)),
 (164, ('about', 202)),
 (402, ('shares', 202)),
 (532, ('may', 202)),
 (1, ('data', 199)),
 (476, ('are', 199)),
 (521, ('euro', 190)),
 (1261, ('it', 186)),
 (90, ('ecb', 183)),
 (193, ('not', 182)),
 (1012, ('out', 178)),
 (253, ('that', 175)),
 (1725, ('sales', 172)),
 (623, ('fed', 170)),
 (1592, ('deal', 166)),
 (813, ('you', 